In [144]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import keras
import h5py
import PIL
import seaborn as sns
import plotly
import sklearn_pandas
from IPython.display import Image
from IPython.core.display import HTML 
#from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn_pandas import DataFrameMapper
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer

In [145]:
# Esta celda lo que hace es obtener el dataset que se encuentra en el repositorio, utilizando un path relativo
path = '..//TP2//Seasons_Stats.csv'
data = pd.read_csv(path)

In [146]:
# Con esta celda buscamos reducir la cantidad de columnas que posee el dataset, eliminando las columnas que pensamos no son necesarias para realizar la predicción.
columns = ['blanl','blank2', 'Tm']
data = data.drop(columns, axis=1)

In [147]:
# Eliminamos todas las filas de datos que sean anteriores al año 1980. Elegimos este año ya que a partir de aquí se comienzan a tener en cuenta estadísticas como, por ejemplo, los tiros de 3 puntos.
# Esto lo hacemos ya que, antes de ese año, el dataset poseía muchas estadísticas que todavía no se tenian en cuenta y había muchos valores en NaN
data = data.drop(data[data.Year < 1980].index, axis=0)
# Además, eliminamos todos los datos en NaN para evitar inconsistencias en el dataset.
#data = data.dropna()

#Con esta linea mostramos todas las columnas del dataset
pd.set_option('display.max_columns', 100)

In [148]:
df = pd.DataFrame(data)
display(df)

,Unnamed: 0,Year,Player,Pos,Age,G,GS,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
312,312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
487,487,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
618,618,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
779,779,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
911,911,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24686,24686,2017.0,Cody Zeller,PF,24.0,62.0,58.0,1725.0,16.7,0.604,0.002,0.442,8.6,17.3,12.9,9.1,1.8,3.0,10.9,15.5,3.4,2.2,5.6,0.157,-0.2,2.3,2.1,1.8,253.0,443.0,0.571,0.0,1.0,0.000,253.0,442.0,0.572,0.571,133.0,196.0,0.679,135.0,270.0,405.0,99.0,62.0,58.0,65.0,189.0,639.0
24687,24687,2017.0,Tyler Zeller,C,27.0,51.0,5.0,525.0,13.0,0.508,0.006,0.247,9.2,17.0,13.2,12.2,0.7,3.3,10.2,16.5,0.5,0.6,1.0,0.094,-3.2,0.8,-2.5,-0.1,78.0,158.0,0.494,0.0,1.0,0.000,78.0,157.0,0.497,0.494,22.0,39.0,0.564,43.0,81.0,124.0,42.0,7.0,21.0,20.0,61.0,178.0
24688,24688,2017.0,Stephen Zimmerman,C,20.0,19.0,0.0,108.0,7.3,0.346,0.000,0.161,10.8,24.9,17.6,5.3,0.9,3.7,8.3,14.8,-0.1,0.1,0.0,-0.005,-7.8,0.4,-7.3,-0.1,10.0,31.0,0.323,0.0,0.0,NaN,10.0,31.0,0.323,0.323,3.0,5.0,0.600,11.0,24.0,35.0,4.0,2.0,5.0,3.0,17.0,23.0
24689,24689,2017.0,Paul Zipser,SF,22.0,44.0,18.0,843.0,6.9,0.503,0.448,0.181,1.9,14.2,8.0,6.1,0.9,1.5,14.4,14.4,-0.3,0.8,0.5,0.030,-3.6,-0.1,-3.7,-0.4,88.0,221.0,0.398,33.0,99.0,0.333,55.0,122.0,0.451,0.473,31.0,40.0,0.775,15.0,110.0,125.0,36.0,15.0,16.0,40.0,78.0,240.0


In [149]:
df[['3P%','FT%', '2P%', '3PAr', 'FTr', 'FG%', 'eFG%', 'TS%', 'TOV%']] = df[['3P%','FT%', '2P%', '3PAr', 'FTr', 'FG%', 'eFG%', 'TS%', 'TOV%']].fillna(0)

In [150]:
# Este celda elimina los años que figuran mas de una vez para un mismo jugador.
df = df.drop_duplicates(subset=['Player','Year'])

In [151]:
# Esta celda renombra la primer columna del dataFrame (denominada 'Unnamed: 0') y le setea el nombre de "id"
df = df.rename(columns = {'Unnamed: 0' : 'id'})

In [152]:
# Con esta celda, creamos la que será nuestra columna TARGET y la llamamos "morePoints" ("Mas Puntos")
# Dicha columna podrá tener solo dos valores "Yes"/"No" dependiendo si el jugador hizo mas puntos que la temporada anterior
# Cabe recalcar que si es la 1er temporada del jugador o si una temporada no jugo por algun motivo, se colocará en "Yes" ya que consideramos que al no haber jugado efectivamente realizó mas puntos.
df.insert(18, 'morePoints', 'No', allow_duplicates = True)

In [153]:
# Esta es una funcion que realizamos para setear la columna "morePoints"
# Buscamos el año anterior y comparamos los puntos que realizó en ese año y el año actual.
# Si efectivamente hizo mas puntos no hacemos nada, pero si no seteamos la columna en "No"
def funcionSetMorePoints(id):
    auxiliar = df.loc[id]
    temporadaSiguiente = df.loc[(df.Player == auxiliar.Player) & (df.Year == auxiliar.Year  + 1)]      
    if ((not temporadaSiguiente.empty) and (auxiliar.PTS <= temporadaSiguiente.PTS.item())):
        df.loc[df.id == id,'morePoints'] = 'Yes'
    else:
        if temporadaSiguiente.empty:
            df.loc[df.id == id,'morePoints'] = 'None'
    return id

In [154]:
# Llamamos a la funcion defenida anteriormente
df.id = df.id.apply(funcionSetMorePoints)

In [155]:
#Cuando la columna morePoints se encuentra en None quiere decir que es la ultima temporada del jugador
#En este caso se debe eliminar la fila ya que no puede ser utilizada para entrenar los modelos.
df = df[df['morePoints'] != 'None']

In [156]:
df['GS'] = df['GS'].fillna(32.033531)

In [157]:
df = df.dropna()

In [158]:
df = df.round(2)

# TP N° 2 - Experimentación

## Métrica a utilizar

Accuracy<br>
Accuracy nos indica el porcentaje de casos que se acertaron. 
Los problemas que pueden aparecer al utilizar esta métrica no aplicarían para este modelo, ya que: 
•	Como pudimos apreciar en el análisis exploratorio de datos, no poseemos un gran desbalanceo en los datos. 
•	El resultado de la predicción no tiene diferente impacto (tiene igual impacto errar por Si o por No). Creemos que el costo de obtener falsos positivos no es alto. 
Por esta razón consideramos que la métrica Accuracy es la que más se adapta a nuestro caso de estudio.


**No tenemos ninguno de estos dos problemas, para mi podría ser Accuracy ANASHEEEIIII.**

## Técnica de feature engineering

Podría ser Binning (para achicar los decimales). Data pasada creemos que no xq tendríamos que tener los registros de cada jugador al momento de predecir y no los vamos a tener

## Dividiendo el Dataset

In [159]:
df['Pos'] = df['Pos'].str.replace('-',' ')
#jugador = df[df['Player'] == "Andres Nocioni"]
#display(jugador)
columns = ['id','Player','Year']
df = df.drop(columns, axis=1)


,id,Year,Player,Pos,Age,G,GS,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,morePoints,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
17456,17456,2005.0,Andres Nocioni,SF,25.0,81.0,38.0,1892.0,10.1,0.48,0.16,0.37,4.1,18.6,11.4,10.9,1.0,1.3,Yes,16.2,19.7,-0.9,3.3,2.5,0.06,-3.9,1.7,-2.1,-0.1,241.0,601.0,0.40,25.0,97.0,0.26,216.0,504.0,0.43,0.42,170.0,222.0,0.77,69.0,317.0,386.0,122.0,37.0,34.0,135.0,208.0,677.0
18024,18024,2006.0,Andres Nocioni,PF,26.0,82.0,43.0,2239.0,16.1,0.57,0.29,0.29,4.6,21.0,12.8,9.0,0.9,1.8,No,11.3,21.3,3.3,3.4,6.7,0.14,0.2,0.8,1.0,1.7,383.0,831.0,0.46,93.0,238.0,0.39,290.0,593.0,0.49,0.52,204.0,242.0,0.84,89.0,411.0,500.0,117.0,40.0,52.0,120.0,252.0,1063.0
18566,18566,2007.0,Andres Nocioni,PF,27.0,53.0,31.0,1406.0,15.6,0.58,0.36,0.25,3.4,20.9,12.2,7.0,1.0,1.3,Yes,13.8,23.8,1.4,2.6,4.1,0.14,-0.4,-0.2,-0.6,0.5,273.0,584.0,0.47,80.0,209.0,0.38,193.0,375.0,0.52,0.54,123.0,145.0,0.85,42.0,259.0,301.0,57.0,27.0,24.0,104.0,164.0,749.0
19140,19140,2008.0,Andres Nocioni,SF,28.0,82.0,27.0,2021.0,13.6,0.55,0.40,0.28,2.6,17.0,9.5,8.3,0.6,1.7,No,11.9,24.5,1.5,1.8,3.3,0.08,-0.3,-2.1,-2.4,-0.2,379.0,877.0,0.43,126.0,346.0,0.36,253.0,531.0,0.48,0.50,196.0,243.0,0.81,48.0,294.0,342.0,95.0,25.0,43.0,133.0,245.0,1080.0
19730,19730,2009.0,Andres Nocioni,PF SF,29.0,76.0,18.0,1991.0,12.2,0.56,0.44,0.26,3.1,17.8,10.5,8.3,1.0,1.2,No,12.6,20.1,1.4,1.4,2.8,0.07,-0.4,-1.6,-2.0,0.0,299.0,701.0,0.43,124.0,311.0,0.40,175.0,390.0,0.45,0.52,145.0,183.0,0.79,54.0,307.0,361.0,102.0,39.0,33.0,113.0,261.0,867.0
20327,20327,2010.0,Andres Nocioni,SF,30.0,75.0,28.0,1477.0,11.1,0.50,0.42,0.17,2.5,14.9,8.6,8.0,1.1,1.3,No,9.6,20.9,0.4,1.0,1.3,0.04,-1.5,-1.9,-3.4,-0.5,234.0,587.0,0.40,96.0,249.0,0.39,138.0,338.0,0.41,0.48,71.0,99.0,0.72,32.0,190.0,222.0,75.0,32.0,26.0,67.0,160.0,635.0
20925,20925,2011.0,Andres Nocioni,SF,31.0,54.0,17.0,931.0,10.1,0.54,0.43,0.22,2.4,18.6,10.5,7.4,0.8,1.2,No,13.4,17.4,0.2,1.1,1.3,0.07,-2.3,-0.6,-3.0,-0.2,118.0,277.0,0.43,42.0,118.0,0.36,76.0,159.0,0.48,0.50,49.0,61.0,0.80,19.0,150.0,169.0,45.0,14.0,15.0,47.0,100.0,327.0


In [185]:

mapper = DataFrameMapper([
    (['Age'], [StandardScaler()]),
    (['G'], [StandardScaler()]),
    (['GS'], [StandardScaler()]),
    (['MP'], [StandardScaler()]),
    (['PER'], [StandardScaler()]),
    (['ORB%'], [StandardScaler()]),
    (['DRB%'], [StandardScaler()]),
    (['TRB%'], [StandardScaler()]),
    (['AST%'], [StandardScaler()]),
    (['TOV%'], [StandardScaler()]),
    (['OWS'], [StandardScaler()]),
    (['DWS'], [StandardScaler()]),
    (['WS'], [StandardScaler()]),
    (['STL%'], [StandardScaler()]),
    (['BLK%'], [StandardScaler()]),
    (['OWS'], [StandardScaler()]),
    (['OBPM'], [StandardScaler()]),
    (['DBPM'], [StandardScaler()]),
    (['BPM'], [StandardScaler()]),
    (['VORP'], [StandardScaler()]),
    (['USG%'], [StandardScaler()]),
    (['PTS'], [StandardScaler()]),
    (['FG'], [StandardScaler()]),
    (['3P'], [StandardScaler()]),
    (['3PA'], [StandardScaler()]),
    (['2P'],[StandardScaler()]),
    (['2PA'],[StandardScaler()]),
    (['FT'], [StandardScaler()]),
    (['FTA'],[StandardScaler()]),
    (['ORB'],[StandardScaler()]),
    (['DRB'],[StandardScaler()]),
    (['TRB'],[StandardScaler()]),
    (['STL'],[StandardScaler()]),
    (['BLK'],[StandardScaler()]),
    (['TOV'],[StandardScaler()]),
    (['PF'],[StandardScaler()]),
    (['Pos'],[OneHotEncoder()]), 
])


In [186]:
display(mapper)

DataFrameMapper(drop_cols=[],
                features=[(['Age'], [StandardScaler()]),
                          (['G'], [StandardScaler()]),
                          (['GS'], [StandardScaler()]),
                          (['MP'], [StandardScaler()]),
                          (['PER'], [StandardScaler()]),
                          (['ORB%'], [StandardScaler()]),
                          (['DRB%'], [StandardScaler()]),
                          (['TRB%'], [StandardScaler()]),
                          (['AST%'], [StandardScaler()]),
                          (['TOV%'], [StandardScaler()]),
                          (['OWS'], [StandardSc...
                          (['BPM'], [StandardScaler()]),
                          (['VORP'], [StandardScaler()]),
                          (['USG%'], [StandardScaler()]),
                          (['PTS'], [StandardScaler()]),
                          (['FG'], [StandardScaler()]),
                          (['3P'], [StandardScaler()]),
      

In [187]:
#Dejamos un 80% del dataset para entrenar, un 10% para validar y un 10% para testear
train, not_train = train_test_split(df, test_size=0.2, random_state=42)
validation, test = train_test_split(not_train, test_size=0.5,random_state=42)

In [188]:
mapper.fit(train)

DataFrameMapper(drop_cols=[],
                features=[(['Age'], [StandardScaler()]),
                          (['G'], [StandardScaler()]),
                          (['GS'], [StandardScaler()]),
                          (['MP'], [StandardScaler()]),
                          (['PER'], [StandardScaler()]),
                          (['ORB%'], [StandardScaler()]),
                          (['DRB%'], [StandardScaler()]),
                          (['TRB%'], [StandardScaler()]),
                          (['AST%'], [StandardScaler()]),
                          (['TOV%'], [StandardScaler()]),
                          (['OWS'], [StandardSc...
                          (['BPM'], [StandardScaler()]),
                          (['VORP'], [StandardScaler()]),
                          (['USG%'], [StandardScaler()]),
                          (['PTS'], [StandardScaler()]),
                          (['FG'], [StandardScaler()]),
                          (['3P'], [StandardScaler()]),
      

In [189]:
sample = train.sample(5, random_state=42)
mapper.transform(sample)

array([[-0.91921172,  0.45404305, -1.02789316, -0.66022435,  0.45931369,
         1.85308161,  1.43621304,  1.75469548, -0.26612573,  0.17879923,
        -0.15126014, -0.20786501, -0.16017576, -0.08098277,  0.45261107,
        -0.15126014, -0.12234665,  0.83370282,  0.34609345, -0.22148819,
        -0.76688671, -0.74804361, -0.73390244, -0.65034921, -0.68406433,
        -0.62251957, -0.63640034, -0.59258185, -0.5884753 ,  0.53817023,
        -0.0593374 ,  0.13758182, -0.60390559, -0.10189178, -0.69098176,
        -0.29667645,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.10930218,  0.87806906, -0.96123492,  0.1962731 ,  0.45931369,
        -0.48533753, -0.88959306, -0.78826314,  0.34082657, -0.99630292,
         0.9540005 ,  0.41847243,  0.84917985,  0.67359161, -0.41356541,
         0.9540005 ,  0.58595

In [190]:
mapper.transformed_names_

['Age',
 'G',
 'GS',
 'MP',
 'PER',
 'ORB%',
 'DRB%',
 'TRB%',
 'AST%',
 'TOV%',
 'OWS',
 'DWS',
 'WS',
 'STL%',
 'BLK%',
 'OWS',
 'OBPM',
 'DBPM',
 'BPM',
 'VORP',
 'USG%',
 'PTS',
 'FG',
 '3P',
 '3PA',
 '2P',
 '2PA',
 'FT',
 'FTA',
 'ORB',
 'DRB',
 'TRB',
 'STL',
 'BLK',
 'TOV',
 'PF',
 'Pos_x0_C',
 'Pos_x0_C PF',
 'Pos_x0_PF',
 'Pos_x0_PF C',
 'Pos_x0_PF SF',
 'Pos_x0_PG',
 'Pos_x0_PG SF',
 'Pos_x0_PG SG',
 'Pos_x0_SF',
 'Pos_x0_SF PF',
 'Pos_x0_SF SG',
 'Pos_x0_SG',
 'Pos_x0_SG PF',
 'Pos_x0_SG PG',
 'Pos_x0_SG SF']

In [209]:

lr_model_si = Pipeline([
    ('mapper', mapper),
    ('classifier', LogisticRegression(random_state=42,max_iter=300, solver = 'liblinear'))
])

lr_model_si.fit(train, train.morePoints)

lr_model_si['classifier'].coef_

y_pred = lr_model_si.predict(validation)

y_pred

array(['No', 'Yes', 'No', ..., 'No', 'Yes', 'Yes'], dtype=object)

In [210]:
print(metrics.classification_report(validation.morePoints, y_pred))
print(metrics.accuracy_score(validation.morePoints, y_pred))

              precision    recall  f1-score   support

          No       0.66      0.78      0.71       674
         Yes       0.64      0.49      0.56       536

    accuracy                           0.65      1210
   macro avg       0.65      0.64      0.64      1210
weighted avg       0.65      0.65      0.64      1210

0.6520661157024793


### COMO SE VE ACA, ANDA RE BIEN
 #SALUDOS PROFE

AttributeError: 'Pipeline' object has no attribute 'coef_'

In [203]:
lr_model_si['classifier'].coef_

array([[-0.50785427, -0.68224795, -0.14179643,  0.04628022, -0.11648174,
        -0.59185564, -0.73372163,  1.29835237,  0.09882936, -0.00745028,
         0.16084799,  0.13916547,  0.06788318, -0.07862706, -0.03304008,
         0.16084799,  0.31914957,  0.32537711, -0.08984209, -0.2874037 ,
         0.21455277, -0.17655663, -0.19956884, -0.25611849,  0.16820287,
        -0.14998275,  0.25321681, -0.01656239, -0.19019433,  0.00355584,
        -0.06931483, -0.04747496,  0.04205398,  0.13870743,  0.09579055,
        -0.01026029, -0.36667662, -0.57691348, -0.26471124,  0.05708376,
         0.48996696, -0.05802166,  0.57805377, -0.29193441, -0.03063104,
         0.12326776,  0.34869057,  0.04254607, -0.12135902,  0.52934697,
        -0.45962665]])

In [148]:
#esto es para probar el binning, agrupando los PTS en distintos grupos
#data = df.squeeze()
#bins = [0,250,500,750,1000,1250,1500,1750,2000,2250,2500,2750,3000,3250,3500,3750,4000]
#binned_Data = pd.cut(data.PTS, bins)
#display(binned_Data)

5727     (2000, 2250]
5728       (250, 500]
5729     (1000, 1250]
5730     (1000, 1250]
5731         (0, 250]
             ...     
24090        (0, 250]
24091      (250, 500]
24092    (1000, 1250]
24093      (500, 750]
24094      (250, 500]
Name: PTS, Length: 12098, dtype: category
Categories (16, interval[int64]): [(0, 250] < (250, 500] < (500, 750] < (750, 1000] ... (3000, 3250] < (3250, 3500] < (3500, 3750] < (3750, 4000]]

In [149]:
#Cambio la columna PTS por la columna creada.
#df['PTS'] = binned_Data
#display(df)

,id,Year,Player,Pos,Age,G,GS,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,morePoints,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
5727,5727,1980.0,Kareem Abdul-Jabbar*,C,32.0,82.0,32.03,3143.0,25.3,0.64,0.00,0.34,7.2,22.2,15.4,16.5,1.2,4.6,Yes,15.7,24.1,9.5,5.3,14.8,0.23,4.0,2.7,6.7,6.8,835.0,1383.0,0.60,0.0,1.0,0.00,835.0,1382.0,0.60,0.60,364.0,476.0,0.76,190.0,696.0,886.0,371.0,81.0,280.0,297.0,216.0,"(2000, 2250]"
5728,5728,1980.0,Tom Abernethy,PF,25.0,67.0,32.03,1222.0,11.0,0.51,0.00,0.26,5.4,12.0,8.6,9.3,1.4,0.6,No,9.9,13.3,1.2,0.8,2.0,0.08,-1.5,-0.1,-1.6,0.1,153.0,318.0,0.48,0.0,1.0,0.00,153.0,317.0,0.48,0.48,56.0,82.0,0.68,62.0,129.0,191.0,87.0,35.0,12.0,39.0,118.0,"(250, 500]"
5729,5729,1980.0,Alvan Adams,C,25.0,75.0,32.03,2168.0,19.2,0.57,0.00,0.27,8.2,22.4,15.4,21.6,2.3,1.4,No,18.2,21.9,3.1,3.9,7.0,0.16,1.6,2.8,4.4,3.5,465.0,875.0,0.53,0.0,2.0,0.00,465.0,873.0,0.53,0.53,188.0,236.0,0.80,158.0,451.0,609.0,322.0,108.0,55.0,218.0,237.0,"(1000, 1250]"
5730,5730,1980.0,Tiny Archibald*,PG,31.0,80.0,80.00,2864.0,15.3,0.57,0.02,0.55,2.3,5.3,3.8,30.2,1.7,0.2,No,19.7,17.0,5.9,2.9,8.9,0.15,1.1,-1.1,0.0,1.5,383.0,794.0,0.48,4.0,18.0,0.22,379.0,776.0,0.49,0.48,361.0,435.0,0.83,59.0,138.0,197.0,671.0,106.0,10.0,242.0,218.0,"(1000, 1250]"
5731,5731,1980.0,Dennis Awtrey,C,31.0,26.0,32.03,560.0,7.4,0.52,0.00,0.83,6.0,16.9,11.5,9.0,1.0,1.5,Yes,24.8,7.9,0.1,0.5,0.6,0.05,-2.9,1.5,-1.4,0.1,27.0,60.0,0.45,0.0,0.0,0.00,27.0,60.0,0.45,0.45,32.0,50.0,0.64,29.0,86.0,115.0,40.0,12.0,15.0,27.0,66.0,"(0, 250]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24090,24090,2016.0,Joe Young,PG,23.0,41.0,0.00,384.0,9.9,0.43,0.27,0.15,1.7,12.5,7.1,26.9,1.9,0.0,No,15.5,24.4,-0.5,0.5,-0.1,-0.01,-3.9,-1.5,-5.4,-0.3,62.0,169.0,0.37,10.0,46.0,0.22,52.0,123.0,0.42,0.40,20.0,25.0,0.80,6.0,44.0,50.0,65.0,15.0,0.0,33.0,30.0,"(0, 250]"
24091,24091,2016.0,Nick Young,SG,30.0,54.0,2.00,1033.0,9.1,0.48,0.64,0.20,1.4,9.0,5.1,5.4,1.1,0.5,Yes,6.9,18.5,0.1,0.0,0.1,0.00,-1.4,-3.5,-4.8,-0.7,126.0,372.0,0.34,77.0,237.0,0.32,49.0,135.0,0.36,0.44,63.0,76.0,0.83,14.0,83.0,97.0,34.0,23.0,7.0,30.0,50.0,"(250, 500]"
24092,24092,2016.0,Thaddeus Young,PF,27.0,73.0,73.00,2407.0,17.5,0.53,0.03,0.17,8.1,23.0,15.4,9.5,2.3,1.2,No,11.6,21.6,1.9,2.2,4.1,0.08,-0.5,0.6,0.1,1.3,495.0,963.0,0.51,7.0,30.0,0.23,488.0,933.0,0.52,0.52,105.0,163.0,0.64,176.0,484.0,660.0,135.0,112.0,37.0,136.0,182.0,"(1000, 1250]"
24093,24093,2016.0,Cody Zeller,C,23.0,73.0,60.00,1774.0,16.1,0.59,0.02,0.53,8.4,19.7,14.0,6.3,1.6,2.9,Yes,11.2,15.4,3.6,2.7,6.3,0.17,-1.0,2.0,1.0,1.3,231.0,437.0,0.53,1.0,10.0,0.10,230.0,427.0,0.54,0.53,175.0,232.0,0.75,138.0,317.0,455.0,71.0,57.0,63.0,68.0,204.0,"(500, 750]"
